In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import time
from datasets import load_dataset
from PIL import Image
from torchvision import transforms
from typing import Dict, Tuple
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

In [ ]:
# Convert .jsn file to .csv
import json
mscoco_train2014_que = json.load(open(f'/kaggle/input/visual-question-answering/v2_Questions_Train_mscoco/v2_OpenEnded_mscoco_train2014_questions.json','r'))
mscoco_train2014_ans = json.load(open(f'/kaggle/input/visual-question-answering/v2_Annotations_Train_mscoco/v2_mscoco_train2014_annotations.json','r'))

mscoco_train2014_que_df = pd.DataFrame(mscoco_train2014_que['questions'])
mscoco_train2014_ans_df = pd.DataFrame(mscoco_train2014_ans['annotations'])

mscoco_train2014 = pd.merge(mscoco_train2014_que_df, mscoco_train2014_ans_df, on=["image_id", "question_id"])

cols = ['image_id','question_id','question','question_type','multiple_choice_answer','answers','answer_type']
mscoco_train2014 = mscoco_train2014[cols]


#convert image_id to image_path
def image_id_to_path(subset,image_id):
    imdir='/kaggle/input/visual-question-answering/train2014/%s/COCO_%s_%012d.jpg'
    image_path = imdir%(subset, subset, image_id)
    return image_path

subset = 'train2014'
image_path = []
for i in (range(len(mscoco_train2014))):
    image_path.append(image_id_to_path(subset,mscoco_train2014['image_id'][i]))
mscoco_train2014['image_id'] = image_path


#create a list of multiple answers of an question
answers_list = []
for i in (range(len(mscoco_train2014))):
    ans_list = []
    for dic in (mscoco_train2014['answers'][i]):
        ans_list.append(dic['answer'])
    answers_list.append(ans_list)
mscoco_train2014['answers'] = answers_list

# save dataframe to .csv
mscoco_train2014.to_csv("/kaggle/working/mscoco_train2014.csv",index=False)


In [ ]:
mscoco_train2014_que_df

In [ ]:
mscoco_train2014_ans_df

In [ ]:
mscoco_train2014

In [ ]:
mscoco_train2014.shape

In [ ]:
from IPython.display import Image

# Display the image inline
Image(filename=mscoco_train2014['image_id'][12])


In [ ]:
# Group the dataset by image_id and count the number of questions per image
image_counts = mscoco_train2014.groupby('image_id').size().reset_index(name='count')
image_counts

In [ ]:
image_counts.shape

In [ ]:
sampled_image_ids = image_counts.sample(frac=0.25, random_state=42)['image_id']
sampled_image_ids

In [ ]:
sampled_image_ids.shape

In [ ]:
# Filter the dataset to keep only the rows corresponding to the sampled image_ids
train_df = mscoco_train2014[mscoco_train2014['image_id'].isin(sampled_image_ids)]
train_df

In [ ]:
train_df.shape

In [ ]:
train_df.to_csv("/kaggle/working/train_sampled.csv",index=False)

In [ ]:
# Filter rows based on the particular value of the column
filtered_df = train_df[train_df['answer_type'] == 'number']

# Display the filtered DataFrame
filtered_df

In [ ]:
# Get the distinct values from the 'answers' column
distinct_values = (train_df.explode('answers'))['answers'].unique()
distinct_values.shape

In [ ]:
distinct_values = train_df['multiple_choice_answer'].unique()

In [ ]:
distinct_values.shape

In [ ]:
train_df.isna().sum()

In [ ]:
answer = list(train_df['multiple_choice_answer'])

In [ ]:
import string
def find_punctuation(text):
    punctuation_list = []
    for t in text:
        for i in t:
            if i in string.punctuation:  
                punctuation_list.append(i)
    punctuation_list = list(set(punctuation_list))
    return punctuation_list

ans_punct_list = find_punctuation(answer)
print(f'{len(ans_punct_list)} punctuations found in the Answer Dataset:\n{ans_punct_list}')

In [ ]:
# Assuming 'df' is your DataFrame and 'column_name' is the name of the column you want to check

# Split each value in the column into words
words = train_df['multiple_choice_answer'].str.split()

# Filter values that are not one-word
not_one_word_values = train_df.loc[words.apply(len) != 1, 'multiple_choice_answer']

# Print the values that are not one-word
print(not_one_word_values)

In [ ]:
import re
def decontractions(phrase):
    """decontracted takes text and convert contractions into natural form.
     ref: https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python/47091490#47091490"""
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"won\’t", "will not", phrase)
    phrase = re.sub(r"can\’t", "can not", phrase)
    
    phrase = re.sub(r"he\'s", "he is", phrase)
    phrase = re.sub(r"she\'s", "she is", phrase)
    phrase = re.sub(r"it\'s", "it is", phrase)
    
    phrase = re.sub(r"he\’s", "he is", phrase)
    phrase = re.sub(r"she\’s", "she is", phrase)
    phrase = re.sub(r"it\’s", "it is", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)

    phrase = re.sub(r"n\’t", " not", phrase)
    phrase = re.sub(r"\’re", " are", phrase)
    phrase = re.sub(r"\’d", " would", phrase)
    phrase = re.sub(r"\’ll", " will", phrase)
    phrase = re.sub(r"\’t", " not", phrase)
    phrase = re.sub(r"\’ve", " have", phrase)
    phrase = re.sub(r"\’m", " am", phrase)

    return phrase


def text_preprocess(text):
    text = text.lower()
    text = decontractions(text) # replace contractions into natural form
    text = re.sub('[-,:]', ' ', text) # replace the character "-" "," with space
    text = re.sub("(?!<=\d)(\.)(?!\d)", '', text) # remove the character ".", except from floating numbers
    text = re.sub('[^A-Za-z0-9. ]+', '', text) # remove all punctuation, except A-Za-z0-9 
    text = re.sub(' +', ' ', text) # remove extra space
    return text

def text_preprocess_ans(text):
    text = text.lower()
    text = decontractions(text) # replace contractions into natural form
    text = re.sub('[-,:]', ' ', text) # replace the character "-" "," with space
    text = re.sub("(?!<=\d)(\.)(?!\d)", '', text) # remove the character ".", except from floating numbers
    text = re.sub('[^A-Za-z0-9.! ]+', '', text) # remove all punctuation, except A-Za-z0-9 
    text = re.sub(' +', ' ', text) # remove extra space
    return text

In [ ]:
train_df["question_preprocessed"] = train_df["question"].map(lambda x: text_preprocess(x))
train_df["answer_preprocessed"] = train_df["multiple_choice_answer"].map(lambda x: text_preprocess_ans(x))

In [ ]:
train_df

In [ ]:
train_df = train_df.drop(['question', 'multiple_choice_answer'], axis=1)

cols = ['image_id','question_id','question_preprocessed','question_type','answer_preprocessed','answers','answer_type']
train_df = train_df[cols]

train_df

In [ ]:
import matplotlib.pyplot as plt

answer_frequency = train_df['answer_preprocessed'].value_counts()

# Select the top 20 answers based on frequency
top_20_answers = answer_frequency.head(20)

# Plot the distribution for the top 20 answers
plt.figure(figsize=(10, 6))
top_20_answers.plot(kind='bar')
plt.title('Distribution of Top 20 Answers')
plt.xlabel('Answer')
plt.ylabel('Frequency')
plt.show()
print(top_20_answers.sum())

In [ ]:
answer_frequency = train_df['answer_preprocessed'].value_counts()

# Select the top 20 answers based on frequency
top_1000_answers = answer_frequency.head(1000)
top_1000_answers.sum()

In [ ]:
answer_frequency = train_df['answer_preprocessed'].value_counts()

top_1000_answers = answer_frequency.head(1000).index.tolist()

# Filter the dataset to include only rows where the "answer" column contains one of the top 1000 answers
final_train_df = train_df[train_df['answer_preprocessed'].isin(top_1000_answers)]

final_train_df


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Assuming 'df' is your DataFrame and 'multiple_choice_answer' is the name of the column

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the "multiple_choice_answer" column
final_train_df['answer_encoded'] = label_encoder.fit_transform(final_train_df['answer_preprocessed'])


In [ ]:
final_train_df

In [ ]:
final_train_df.describe()

In [ ]:
final_train_df.head(20)

In [ ]:
# # Split each value in the column into words
# word_counts = final_train_df['answer_preprocessed'].str.split().apply(len)

# # Check if all values contain only one word
# all_one_word = (word_counts == 1).all()

# # Print the result
# if all_one_word:
#     print("All values in the column are one-word.")
# else:
#     print("Not all values in the column are one-word.")


In [ ]:
# # Assuming 'df' is your DataFrame and 'column_name' is the name of the column you want to check

# # Split each value in the column into words
# words = final_train_df['answer_preprocessed'].str.split()

# # Filter values that are not one-word
# not_one_word_values = final_train_df.loc[words.apply(len) != 1, 'answer_preprocessed']

# # Print the values that are not one-word
# print(not_one_word_values)


In [ ]:
final_train_df.to_csv("/kaggle/working/textpreprocessed_labelencoding.csv",index=False)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from transformers import BertTokenizer, AutoImageProcessor

class VQADataset(Dataset):
    def __init__(self, dataframe, bert_tokenizer, image_processor):
        self.dataframe = dataframe
        self.bert_tokenizer = bert_tokenizer
        self.image_processor = image_processor

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        
        # Process the text
        text_inputs = self.bert_tokenizer(
            row['question_preprocessed'],
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        # Process the image
        image = Image.open(row['image_id']).convert("RGB")
        image_inputs = self.image_processor(images=image, return_tensors='pt')
        
        # Prepare the label
        label = torch.tensor(row['answer_encoded'], dtype=torch.long)
        
        # Merge the inputs
        inputs = {
            'input_ids': text_inputs['input_ids'].squeeze(0),
            'attention_mask': text_inputs['attention_mask'].squeeze(0),
            'pixel_values': image_inputs['pixel_values'].squeeze(0),
            'labels': label
        }
        return inputs


In [ ]:
train_df, test_df = train_test_split(final_train_df, test_size=0.2, random_state=42)

In [ ]:
print(len(train_df), len(test_df))

In [ ]:
# Initialize tokenizers
bert_tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-uncased')
image_processor = AutoImageProcessor.from_pretrained('google/vit-base-patch16-224')

# Create a dataset instance
train_dataset = VQADataset(train_df, bert_tokenizer, image_processor)
eval_dataset = VQADataset(test_df, bert_tokenizer, image_processor)

# Create a DataLoader instance
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_dataloader = DataLoader(eval_dataset, batch_size=2, shuffle=True)


In [ ]:
import torch.nn as nn
from transformers import BertModel, ViTModel

class VQAModel(nn.Module):
    def __init__(self, bert_model_name='google-bert/bert-base-uncased', vit_model_name='google/vit-base-patch16-224', num_labels=1000, intermediate_dim=128):
        super(VQAModel, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.vit = ViTModel.from_pretrained(vit_model_name)
        self.fusion = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size + self.vit.config.hidden_size, intermediate_dim),
            nn.ReLU(),
            nn.Dropout(0.5),
        )
        
        self.classifier = nn.Linear(intermediate_dim, num_labels)
        # self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask, pixel_values, labels=None):
        # Get BERT and ViT outputs
        bert_outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        vit_outputs = self.vit(pixel_values=pixel_values)
        
        fused_output = self.fusion(
            torch.cat(
                [
                    bert_outputs.pooler_output,
                    vit_outputs.pooler_output,
                ],
                dim=1
            )
        )
        
        logits = self.classifier(fused_output)
        
        # Element-wise dot product of BERT and ViT embeddings
#         combined = bert_outputs.last_hidden_state[:, 0, :] * vit_outputs.pooler_output
        
        # Classification
#         logits = self.classifier(combined)
        
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits, labels)
            return loss, logits
        else:
            return logits


In [ ]:
def compute_metrics(eval_tuple: Tuple[np.ndarray, np.ndarray]) -> Dict[str, float]:
    logits, labels = eval_tuple
    preds = logits.argmax(axis=-1)
    return {
        "acc": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average='weighted'),
        "precision": precision_score(labels, preds, average='weighted'),
        "recall": recall_score(labels, preds, average='weighted'),
    }

In [ ]:
from transformers import Trainer, TrainingArguments

model = VQAModel()
model.to(device)

# Define the training arguments
training_args = TrainingArguments(
    output_dir='/kaggle/working',
    num_train_epochs=2,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='/kaggle/working',
    save_steps=19378,
    save_total_limit=2,
    logging_steps=19378,
    report_to=[],  # Disable logging to wandb
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  # assuming you have a validation dataset
    compute_metrics=compute_metrics,  # You can define metrics if needed
)

# Track time taken for training
start_time = time.time()
train_result = trainer.train()
end_time = time.time()

train_time = end_time - start_time
print(f"Training time: {train_time} seconds")

# # Save the final model
# trainer.save_model('/kaggle/working/vqa_model')

# # Save the training arguments
# training_args.save('/kaggle/working/vqa_model/training_args.bin')

# for epoch in range(training_args.num_train_epochs):
#     print(f"Epoch {epoch}, Loss: {compute_loss(model, train_dataset)}")


In [ ]:
eval_result = trainer.evaluate()

In [ ]:
print(f"Evaluation Metrics: {eval_result}")

In [ ]:
print(f"Training Metrics: {train_result.metrics}")